In [1]:
from elasticsearch_dsl import Mapping, String, Nested, connections, DocType, Object, Search, Index, A, F, Q, Integer
from elasticsearch import Elasticsearch, connection

connections.connections.add_connection('default', Elasticsearch())

In [17]:
index = Index('samples')
index.delete()
index.create()

In [18]:
# index = Index('samples')

# index.create()
m = Mapping('sample')
m.field('sample_id', Integer())
m.field('series_ids', Integer())
m.field('platform_id', Integer())
m.field('extracted_molecule', String(index='not_analyzed'))
m.field('sample_type', String(index='not_analyzed'))
m.field('organism', String(index='not_analyzed'))
m.field('label', String())
m.field('characteristics_raw', String())
# m.field('characteristics', Object())
m.field('source_name', String())
m.field('title', String())
m.save('samples')

In [19]:
# class Sample(DocType):
#     class Meta:§
#         index = 'samples'
        
#     accession = String(index='not_analyzed')
#     series = String(index='not_analyzed')
#     platform = String(index='not_analyzed')
#     extracted_molecule = String(index='not_analyzed')
#     sample_type = String(index='not_analyzed')
#     organism = String(index='not_analyzed')
    
#     label = String()
#     characteristics_raw = String()
#     characteristics_raw = Object()
#     source_name = String()
#     characteristics = Object()
#     title = String()


## Import 

In [20]:
def map_bins(map_func, collection, bin_size):
    bin_ = []
    for item in map(map_func, collection):
        bin_.append(item)
        if len(bin_) >= bin_size:
            yield bin_
            bin_ = []
    yield bin_

In [21]:
data_file = '/Users/nikita/Data/snapshots/mongo/scraper_test_dev/samples.scraper_test_dev.json'
import json
from elasticsearch.helpers import bulk
from itertools import islice
es = Elasticsearch()

fields = list(m.to_dict()['sample']['properties'].keys())

def scrap_fields(d):
    d['sample_id'] = int(d['accession'][3:])
    d['platform_id'] = int(d['platform'][3:])
    d['series_ids'] = [int(x[3:]) for x in d['series']]
    del d['characteristics']
    return d

cut = lambda xs: islice(xs, 1000)
cut = lambda xs: xs

for bucket in map_bins(lambda text: scrap_fields(json.loads(text)), cut(open(data_file)), bin_size=1000):
    actions = [dict(_index='samples', _type='sample', _source=x) for x in bucket if x.get('channel') != 2]
    bulk(es, actions)
    
    

In [26]:
fields = []
for sample in map(json.loads, open(data_file)):
    if sample.get('channel') != 2:
        chars = sample.get('characteristics')
        if chars:
            for f in chars.keys():
                fields.append(f)

In [27]:
import pandas as pd
d = pd.DataFrame(dict(fields=fields))
d['fields'] = d.fields.str.lower()
d.fields.value_counts()

tissue                          484195
unnamed_1                       440234
gender                          254910
age                             240155
strain                          210408
cell type                       209658
treatment                       122071
unnamed_2                       116174
sex                             108985
cell line                       107949
disease state                    80845
genotype                         75401
time                             67267
unnamed_3                        57307
genotype/variation               53598
developmental stage              45426
sample type                      43929
unnamed_4                        39654
dose                             31465
stage                            28539
unnamed_5                        26609
time point                       24929
ethnicity                        21799
group                            21278
vehicle                          21068
unnamed_6                

In [9]:
def aslist(search):
    return list(s._d_ for s in search.execute())

In [115]:
s = (Search(es, index='samples') 
#      .query('match', title='Spleen vs fMLP')
#      .filter('term', organism='Mus musculus')
#      .filter('term', series='GSE4107')
#      .filter('term', platform='GPL263')

     )
    
aslist(s[:10])

[{'_id': {'$oid': '55db51c9a138572c9e9cec25'},
  'accession': 'GSM93927',
  'characteristics': {'Age': '44', 'Gender': 'M', 'Tissue': 'Mucosa'},
  'characteristics_raw': ['Gender:M', 'Age:44', 'Tissue: Mucosa'],
  'data_source': 'geo',
  'description': ['DNA chip technology was used to systematically search for genes differentially expressed in early onset CRC. We report a panel of 7 over-expressed genes in the normal appearing colonic mucosa of young patients without dominant family history compared to that of HC that could potentially predict early onset CRC.'],
  'extracted_molecule': 'total RNA',
  'geo_id': 'GSM93927',
  'instrument_model': None,
  'label': 'fluorescent',
  'last_update_date': {'$date': '2015-01-14T00:00:00.000Z'},
  'library_selection': None,
  'library_source': None,
  'library_strategy': None,
  'meta': {'geo_id': 93927},
  'organism': 'Homo sapiens',
  'platform': 'GPL570',
  'protocols': [{'text': 'GCOS software, MAS5, scaled to 500',
    'type': 'data_proces

In [106]:
from pprint import pprint
from json import dumps

def phrase(ph, fields):
    _q = {'query': ph, 'slop': 1}
    return Q('bool', should=[Q('match_phrase', **({f: _q})) for f in fields])
s = (Search(index='samples') 
        .query(phrase('brain', ['title', 'source_name']) & phrase('normal', ['title', 'source_name'])) 
#         .filter('term', organism='Homo sapiens')
#         .filter('term', platform='GPL2986')
        .filter(F('term', platform_id=2986))
     )
# print(dumps(s.to_dict(), indent=2))
s.execute().hits.total


In [144]:
s = (Search(index='samples') 
        .query(phrase('brain', ['title', 'source_name']) & phrase('normal', ['title', 'source_name'])) 
#         .filter('term', organism='Homo sapiens')
#         .filter('term', platform='GPL2986')
        .filter(F('term', platform_id=2986))
     )
# print(dumps(s.to_dict(), indent=2))
s.execute().hits.total


124

In [145]:
len(aslist(s))

10

In [146]:
import pandas as pd
d = pd.DataFrame(aslist(s[:130]))
d[:1]

,_id,accession,characteristics,characteristics_raw,data_source,description,extracted_molecule,geo_id,instrument_model,label,...,relations,sample_type,scrap_date,series,series_ids,source_name,status,submission_date,supplementary_files,title
0,{'$oid': '55db570ba138572c9e9e3ee5'},GSM160863,{'unnamed_1': 'African Green Monkey cells (chl...,[African Green Monkey cells (chlorocebus sabae...,geo,[Brain from 1 AGM.],total RNA,GSM160863,None,Chemiluminescence / DIG-UTP,...,{},RNA,{'$date': '2015-08-24T17:40:23.569Z'},[GSE6982],[6982],Brain tissue,Public on Oct 01 2007,{'$date': '2007-02-07T00:00:00.000Z'},[],Brain tissue AGM_brain


In [142]:
d.columns

Index(['_id', 'accession', 'characteristics', 'characteristics_raw',
       'data_source', 'description', 'extracted_molecule', 'geo_id',
       'instrument_model', 'label', 'last_update_date', 'library_selection',
       'library_source', 'library_strategy', 'meta', 'organism', 'platform',
       'platform_id', 'protocols', 'relations', 'sample_type', 'scrap_date',
       'series', 'series_ids', 'source_name', 'status', 'submission_date',
       'supplementary_files', 'title'],
      dtype='object')

In [147]:
d[['accession', 'title', 'description', 'series', 'label', 'organism', 'sample_type', 'source_name']]

,accession,title,description,series,label,organism,sample_type,source_name
0,GSM160863,Brain tissue AGM_brain,[Brain from 1 AGM.],[GSE6982],Chemiluminescence / DIG-UTP,Chlorocebus sabaeus,RNA,Brain tissue
1,GSM194472,Human brain replicate 2 of 3,[Human brain replicate 2 of 3],[GSE7905],Digoxigenin-UTP,Homo sapiens,RNA,Clontech Human brain
2,GSM194474,Human fetal brain replicate 1 of 3,[Human fetal brain replicate 1 of 3],[GSE7905],Digoxigenin-UTP,Homo sapiens,RNA,Clontech Human fetal brain
3,GSM194473,Human brain replicate 3 of 3,[Human brain replicate 3 of 3],[GSE7905],Digoxigenin-UTP,Homo sapiens,RNA,Clontech Human brain
4,GSM194475,Human fetal brain replicate 2 of 3,[Human fetal brain replicate 2 of 3],[GSE7905],Digoxigenin-UTP,Homo sapiens,RNA,Clontech Human fetal brain
5,GSM194471,Human brain replicate 1 of 3,[Human brain replicate 1 of 3],[GSE7905],Digoxigenin-UTP,Homo sapiens,RNA,Clontech Human brain
6,GSM194476,Human fetal brain replicate 3 of 3,[Human fetal brain replicate 3 of 3],[GSE7905],Digoxigenin-UTP,Homo sapiens,RNA,Clontech Human fetal brain
7,GSM291582,Normal collateral EC gene expression N2,[Transriptional profiling was performed with e...,[GSE11583],Digoxigenin,Homo sapiens,RNA,"human umbilical vein endothelial cells, normal..."
8,GSM291584,Normal collateral EC gene expression N3,[Transcriptional profiling was performed with ...,[GSE11583],Digoxigenin,Homo sapiens,RNA,"human umbilical vein endothelial cells, normal..."
9,GSM291580,Normal collateral EC gene expression N1,[Transriptional profiling was performed with e...,[GSE11583],Digoxigenin,Homo sapiens,RNA,"human umbilical vein endothelial cells, normal..."
